# Análisis de certificados en Catastro

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />

La división de gobierno digital necesita crear un método que identifique y clasifique de manera automática los trámites que están disponibles en la página web www.chileatiende.cl

En este notebook veremos como usar pandas para cargas archivos excel e identificar los trámites que son del tipo certificados


In [ ]:
import os
import pandas as pd
import numpy as np
import json
from itertools import groupby

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Cargamos en memoria el archivo de Catastro

In [ ]:
catastro_file="data/Catastro 23082018 V4.xlsx"
catastro_df=pd.read_excel(catastro_file)
catastro_df.head()

In [ ]:
catastro_df.columns

In [ ]:
len(catastro_df)

In [ ]:
len(catastro_df['ID trámite'].unique())

In [ ]:
len(catastro_df['Nombre del trámite'].unique())

In [ ]:
with pd.option_context('display.max_colwidth', 300):
    print(catastro_df.loc[catastro_df['Nombre del trámite'].str.contains('nacimiento'), ['ID trámite','Nombre del trámite']])

# 2. Extraemos las columnas de interes

In [ ]:
columns=['ID trámite','Nombre del trámite','Requisitos']

data_df = catastro_df[columns].copy()
data_df.rename(index=int, columns={'ID trámite': 'id_tramite', 'Nombre del trámite': 'nombre_tramite', 'Requisitos': 'requisitos'}, inplace=True)
data_df.head()

Convertimos el campo ID trámite a un tipo de datos string

In [ ]:
data_df['id_tramite'] = data_df['id_tramite'].astype(str)

In [ ]:
data_df['LEN_id_tramite'] = data_df['id_tramite'].str.len()

Ordenamos por el largo del campo id_tramite para identificar posibles errores en el tipeo

In [ ]:
data_df.sort_values('LEN_id_tramite').head()

Indentificamos los registros con ID trámite NULL

In [ ]:
data_df[data_df['id_tramite'].isnull()]

Indentificamos los registros con ID trámite = NUEVO

In [ ]:
data_df[data_df['id_tramite'].str.contains('NUEVO')].sort_values('id_tramite')

Copiamos el index en un campo llamado row_excel y sumamos el valor 2

## 3. Limpiamos y parseamos el campo requisitos

In [ ]:
import re
import unicodedata

def standardize_string(row):
    result = row.lower()
    result = unicodedata.normalize('NFD', result)
    result = result.encode('ascii', 'ignore')
    result = result.decode("utf-8")
    return result

def parse_string(row):
    #result = re.split('-|\n', row)
    #result = list(map(lambda x: re.sub('\n$', '', x), result))
    result = re.split('-', row)
    result = list(map(lambda x: x.replace('\n', ' '), result))
    result = list(map(lambda x: re.sub(' +', ' ', x), result))
    result = list(map(str.strip, result))
    result = list(filter(None, result))
    return result

def parse_string_new_line(row):
    result = re.split('\n', row)
    result = list(map(str.strip, result))
    result = list(filter(None, result))
    return result

def remove_numbering(row):
    result = list(map(lambda x: re.sub('^[0-9]+\.', '', x), row))
    result = list(map(str.strip, result))
    return result
    
def remove_stopwords(row):
    stop_spanish = stopwords.words('spanish')
    
    result = ' '.join([word for word in row])
    result = ' '.join([word for word in result.split() if word not in stop_spanish])
    return result

def get_max_length(row):
    result=[]
    for word in row:
        result.append(len(word))
    result = max(result)
    return result

def get_number_new_line(row):
    result=0
    for word in row:
        result += len(re.findall("\n", word))
    return result

def get_hyphens_numbers(row):
    result=re.findall("\w+.{,2}(?:[0-9]+|[a-zA-Z]+)-(?:[0-9]+).{,2}\w+", row)
    return result

def get_hyphens(row):
    result=re.findall("\w+.{,2}\w+-\w+.{,2}\w+", row)
    return result

### Identificamos las palabras que estan unidas por guiones y reemplazamos - por _ para estos pocos casos

In [ ]:
data_df['requisitos_clean'] = data_df['requisitos'].apply(standardize_string)

In [ ]:
#temp_df=data_df['requisitos_clean'].apply(get_hyphens)
#temp_df=temp_df[temp_df.str.len() > 0]
#for idx, row in temp_df.iteritems():
#    print(idx, row)

In [ ]:
hyphens_list1=['te-4','formulario sl-1','datum wgs-84','900.183-2','v-5','901265-6','fpi-50',
               'o-71/026','o-73/002','certificado f-30','mp3-256 kbps','formulario 30-1','gr -g-03','weg-84',
               'w3-article-4833', 'articles-4833_recurso']

In [ ]:
hyphens_list2=['e-declarador','chile-espana','ipso-jure','visa-de-residencia',
               'solicitud-reconcideracio','81n-visa','permanencia-definitiva','directores-accionistas',
               'teorico-practico','on-line','re-exportacion','formulario-tipo','sub-rol','compra-venta',
               'empresa-individual','ex-cora','poblacion-objetivo','carta-compromiso','ips-chileatiende',
               'im-ponentes','anos-calendario','teorica-metodologica','tele-vigilancia','mai-ges','in-situ',
               'y de-los','urbano-arquitectonico','documentacion- del ano en curso-que','blu-ray','bio-bibliografica',
               'post-doctorados','cd-rom','ingreso-sai-v2','declara-cion','en-tidad','anexo n 8-a','imp-mp3',
               're-circulacion','inn-sernapesca','co-responsables']

In [ ]:
hyphens_list = hyphens_list1 + hyphens_list2
hyphens_dict = {x: x.replace('-', '_') for x in hyphens_list}

In [ ]:
for key, val in hyphens_dict.items():
    data_df['requisitos_clean'] = data_df['requisitos_clean'].str.replace(key, val, regex=False)

### Hacemos un parsing del campo requisitos_clean

In [ ]:
data_df['LEN_requisitos_clean'] = data_df['requisitos_clean'].str.len()

In [ ]:
data_df['N_NEW_LINE_requisitos_clean'] = data_df['requisitos_clean'].apply(get_number_new_line)

In [ ]:
data_df['requisitos_parse'] = data_df['requisitos_clean'].apply(parse_string)

In [ ]:
data_df['requisitos_parse'] = data_df['requisitos_parse'].apply(remove_numbering)

In [ ]:
data_df['MAX_LEN_requisitos_parse'] = data_df['requisitos_parse'].apply(get_max_length)

In [ ]:
data_df['N_requisitos_parse'] = data_df['requisitos_parse'].str.len()

In [ ]:
data_df.head()

In [ ]:
data_df.loc[data_df['id_tramite']=='SINID778']

### Buscamos registros que contengan un gran numero de saltos de linea

In [ ]:
temp_df = data_df.sort_values('N_requisitos_parse', ascending=True)
temp_df['ratio_N_NEW_LINE'] = temp_df['N_NEW_LINE_requisitos_clean']/temp_df['N_requisitos_parse']

columns=['id_tramite','MAX_LEN_requisitos_parse','N_requisitos_parse','N_NEW_LINE_requisitos_clean','ratio_N_NEW_LINE']
temp_df = temp_df[temp_df['N_requisitos_parse']<=10][columns]
temp_df = temp_df.sort_values('ratio_N_NEW_LINE', ascending=False).head(30)
temp_df

In [ ]:
data_df.loc[2896,'requisitos_parse']

In [ ]:
if not os.path.exists('results'):
    os.mkdir('results')

output_file='results/catastro_tramites - revisar_saltos_linea_requisitos.xlsx'

writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
temp_df.to_excel(writer, sheet_name='Sheet1', index=False)

workbook  = writer.book
worksheet = writer.sheets['Sheet1']

worksheet.set_column('A:E', 20)

end_row = len(temp_df.index)
end_column = len(temp_df.columns)-1
header = [{'header': word} for word in temp_df.columns.tolist()]

worksheet.add_table(0, 0, end_row, end_column, {'header_row': True, 'columns': header})
writer.save()

### Ordenamos los registros por el largo del campo requisitos_clean - LEN_requisitos_clean

In [ ]:
data_df.sort_values('LEN_requisitos_clean', ascending=True).head()

In [ ]:
data_df.sort_values('LEN_requisitos_clean', ascending=False).head()

Revisamos el registro 4173, el cual tiene un largo de 10741 caracteres el campo requisitos y solo fue parseado en 17 elementos

In [ ]:
data_df.loc[4173, 'requisitos_parse']

### Ordenamos los registros por el numero de elementos del campo requisitos_parse - N_requisitos_parse

In [ ]:
data_df.sort_values('N_requisitos_parse', ascending=True).head()

In [ ]:
data_df.sort_values('N_requisitos_parse', ascending=False).head()

In [ ]:
data_df.sort_values('N_requisitos_parse', ascending=False).head(100).to_excel('results/catastro_tramites - N_requisitos_parse.xlsx', 
            startcol=0,
            startrow=0)

Revisamos el registro con indice 2893, que contiene 48 parrafos y el parrafo mas largo tiene 1427 caracteres

In [ ]:
data_df.loc[2893, 'requisitos_clean']

In [ ]:
data_df.loc[2893, 'requisitos_parse']

### Ordenamos los registros por el maximo largo de los elementos del campo requisitos_parse - MAX_LEN_requisitos_parse

In [ ]:
data_df.sort_values('MAX_LEN_requisitos_parse', ascending=True).head()

In [ ]:
data_df.sort_values('MAX_LEN_requisitos_parse', ascending=False).head()

In [ ]:
data_df.sort_values('MAX_LEN_requisitos_parse', ascending=False).head(100).to_excel('results/catastro_tramites - MAX_LEN_requisitos_parse.xlsx', 
            startcol=0, 
            startrow=0)

# 4. Expandimos el campo requisitos_parse

In [ ]:
data_df.head(10)

In [ ]:
requisitos_df = (pd.melt(data_df['requisitos_parse'].apply(pd.Series).reset_index(), 
             id_vars='index',
             value_name='requisitos_expand').rename_axis('idx')
                 .drop('variable', axis=1)
                 .dropna()
                 .sort_values(['index','idx'])
                 .set_index('index'))

In [ ]:
requisitos_df.head(10)

In [ ]:
data_expand_df = data_df[['id_tramite','nombre_tramite','N_requisitos_parse']].join(requisitos_df, how='outer')
data_expand_df = data_expand_df[['id_tramite','nombre_tramite','requisitos_expand','N_requisitos_parse']]

In [ ]:
data_expand_df.head(10)

In [ ]:
data_expand_df.to_excel('results/catastro_tramites - requisitos_expand.xlsx', 
            startcol=0, 
            startrow=0, 
            index=True)

In [ ]:
#assert false

## 4.1 Creamos columnas para contar numero de caracteres y medir complejidad

In [ ]:
col = data_expand_df['requisitos_expand'].apply(len)
col.name = 'LEN_requisitos_str'
data_expand_df.insert(data_expand_df.columns.get_loc('N_requisitos_parse') + 1, col.name, col)
data_expand_df.head()

In [ ]:
group_df = data_expand_df.groupby('id_tramite')
len(group_df)

In [ ]:
def complexity(row):
    result = max(row['N_requisitos_parse']) * max(row['LEN_requisitos_str'])
    return result

complex_s = group_df.apply(complexity)
#group_s.rename('complexity')

complex_df = pd.DataFrame(complex_s, columns=['complexity']).reset_index()
data_complexity_df = data_expand_df.merge(complex_df, on='id_tramite', how='outer')
data_complexity_df.head()

Dado que calcularemos el logaritmo de be_complexity, convertimos todos los valores 0 a 1

In [ ]:
data_complexity_df.loc[data_complexity_df['complexity']==0,'complexity']=1

Hacemos algunos graficos para entender el comportamiento

In [ ]:
group_df=data_complexity_df.groupby('id_tramite').max()
group_df.sort_values('complexity', ascending=False).head()

In [ ]:
group_df['complexity'].plot(kind='hist', logy=True)

In [ ]:
max_val=max(np.log(group_df['complexity']))
data_complexity_df['complexity_percentage'] = np.round(100*np.log(data_complexity_df['complexity'])/max_val,1)

group_df=data_complexity_df.groupby('id_tramite').max()

In [ ]:
group_df.sort_values('complexity_percentage', ascending=False).head(30)

In [ ]:
output_file='results/catastro_requisitos_complexity.xlsx'
data_df = data_complexity_df.rename_axis('idx').sort_values(['complexity_percentage','idx'], ascending=[False,True])

writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
data_df.to_excel(writer, sheet_name='Sheet1', index=False)

workbook  = writer.book
worksheet = writer.sheets['Sheet1']

format1 = workbook.add_format({'text_wrap': True})

worksheet.set_column('A:A', 10)
worksheet.set_column('B:C', 50, format1)
worksheet.set_column('D:G', 20)

end_row = len(data_df.index)
end_column = len(data_df.columns)-1
header = [{'header': word} for word in data_df.columns.tolist()]

worksheet.add_table(0, 0, end_row, end_column, {'header_row': True, 'columns': header})
writer.save()

In [ ]:
#assert false

# 5. Cargamos en memoria el listado de certificados identificados por la Segpres

In [ ]:
segpres_certificados_file='data/Listado certificados - compartido a Metricarts.xlsx'
segpres_certificados_sheetname='Listado CERTIFICADOS'
segpres_certificados_df=pd.read_excel(segpres_certificados_file, sheet_name=segpres_certificados_sheetname)
segpres_certificados_df.head()

In [ ]:
segpres_certificados_df['nombre_certificado'] = segpres_certificados_df['Nombre del trámite'].apply(standardize_string)

In [ ]:
segpres_certificados_df.sort_values('nombre_certificado', inplace=True)

In [ ]:
segpres_certificados_df.head()

Determinamos el numero de veces que se repiten los nombres de los certificados

In [ ]:
temp_df=segpres_certificados_df.groupby('nombre_certificado').size().reset_index(name='count')
temp_df.sort_values('count', ascending=False).head()

In [ ]:
segpres_certificados_list = segpres_certificados_df['nombre_certificado'].unique()

In [ ]:
segpres_certificados_dict = { name : [name] for name in segpres_certificados_list }

In [ ]:
j = json.dumps(segpres_certificados_dict, indent=2)
f = open('results/certificados_segpres.json', 'w')
print(j, file=f)
f.close()

In [ ]:
#with open("results/certificados_segpres.json") as test:
#    data = test.read()
#    test_dict = json.loads(data)

In [ ]:
#test_dict["certificado anual de estudios"]

In [ ]:
len(segpres_certificados_list)

In [ ]:
segpres_certificados_list[:10]

# 6. Análisis del campo requisitos_expand y creación de catastro_tramites - certificados

Abrir terminal e instalar los siguientes paquetes

`pip install fuzzywuzzy`

`pip install python-Levenshtein`

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import matplotlib.pyplot as plt

In [ ]:
# En caso que queramos volver a cargar en memoria el modulo fuzz

#import importlib
#importlib.reload(fuzz)

Filtramos solamente los registros que contienen la palabra certificado

In [ ]:
data_expand_certificados_df = data_expand_df[data_expand_df['requisitos_expand'].str.contains("certificado")]

In [ ]:
data_expand_certificados_df.head()

In [ ]:
len(data_expand_certificados_df)

In [ ]:
temp_group = data_expand_certificados_df.groupby('id_tramite')
len(temp_group)

### Generación de DataFrame usando lista de certificados de Gobierno Digital e identificados por Metric. Los certificados fueron contetanados en un solo string y registro

In [ ]:
data_expand_certificados_group = data_expand_certificados_df.groupby('id_tramite')
n_group = len(data_expand_certificados_group)

data_tramites_certificados_list=[]
for i, (name, group) in enumerate(data_expand_certificados_group):
    requisitos_expand_list = list(group['requisitos_expand'])
    
    if i % 50 == 0:
        print("Processing {} of {} - ID tramite: {}".format(i, n_group, name))
    
    certificados_match_segpres = []
    for requisito in requisitos_expand_list:
        #print(requisito)
        fuzz_match = process.extract(requisito, segpres_certificados_list, limit=2, scorer=fuzz.partial_ratio)
        #print(fuzz_match)
        for word in fuzz_match:
            if word[1]>90:
                certificados_match_segpres.append(word[0])
    certificados_match_segpres = list(set(certificados_match_segpres))
    
    #print('requisitos_expand_list: ', requisitos_expand_list)
    certificados_match_metric = []
    for requisito in requisitos_expand_list:
        #re_match = re.search('certificado\S?\s+\w+\s+\w+', requisito)
        re_match = re.findall('certificado.*?(?=certificado|$)', requisito)
        for word in re_match:
            # We match a phrase starting with the word certificado and add a maximum of 8 words
            re_search=re.search('certificado(s)?(?:\W+\w+){,8}', word)
            if re_search:
                temp=re_search.group()
                temp=re.split('\.|,|:|\(|\)', temp)[0]
                temp=temp.strip()
                certificados_match_metric.append(temp)
    #print('certificados_match_metric: ', certificados_match_metric)
    certificados_match_metric = list(set(certificados_match_metric))
    
    certificados_match_segpres_string = "\n".join([word for word in certificados_match_segpres])
    certificados_match_metric_string = "\n".join([word for word in certificados_match_metric])
    
    row = {'id_tramite': name,
            'nombre_tramite': group['nombre_tramite'].values[0],
            'certificados_segpres': certificados_match_segpres_string,
            'certificados_metric': certificados_match_metric_string}

    data_tramites_certificados_list.append(row)    

In [ ]:
data_tramites_certificados_df = pd.DataFrame(data_tramites_certificados_list, columns=['id_tramite','nombre_tramite','certificados_segpres','certificados_metric'])
data_tramites_certificados_df.head()

In [ ]:
len(data_tramites_certificados_df)

In [ ]:
output_file='results/catastro_tramites - certificados_segpres_AND_metric.xlsx'

writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
data_tramites_certificados_df.to_excel(writer, sheet_name='Sheet1', index=False)

workbook  = writer.book
worksheet = writer.sheets['Sheet1']

format1 = workbook.add_format({'text_wrap': True})

worksheet.set_column('A:A', 10)
worksheet.set_column('B:D', 50, format1)

end_row = len(data_tramites_certificados_df.index)
end_column = len(data_tramites_certificados_df.columns)-1
header = [{'header': word} for word in data_tramites_certificados_df.columns.tolist()]

worksheet.add_table(0, 0, end_row, end_column, {'header_row': True, 'columns': header})
writer.save()

### Generación de DataFrame usando lista de certificados identificados por Metric y separados en múltiples registros

In [ ]:
data_expand_certificados_group = data_expand_certificados_df.groupby('id_tramite')
n_group = len(data_expand_certificados_group)

data_tramites_certificados_list=[]
for i, (name, group) in enumerate(data_expand_certificados_group):
    requisitos_expand_list = list(group['requisitos_expand'])
    
    if i % 50 == 0:
        print("Processing {} of {} - ID tramite: {}".format(i, n_group, name))
    
    certificados_match_segpres = []
    for requisito in requisitos_expand_list:
        #print(requisito)
        fuzz_match = process.extract(requisito, segpres_certificados_list, limit=2, scorer=fuzz.partial_ratio)
        #print(fuzz_match)
        for word in fuzz_match:
            if word[1]>90:
                certificados_match_segpres.append(word[0])
    certificados_match_segpres = list(set(certificados_match_segpres))
    
    #print('requisitos_expand_list: ', requisitos_expand_list)
    certificados_match_metric = []
    for requisito in requisitos_expand_list:
        #re_match = re.search('certificado\S?\s+\w+\s+\w+', requisito)
        re_match = re.findall('certificado.*?(?=certificado|$)', requisito)
        for word in re_match:
            # We match a phrase starting with the word certificado and add a maximum of 8 words
            re_search=re.search('certificado(s)?(?:\W+\w+){,8}', word)
            if re_search:
                temp=re_search.group()
                temp=re.split('\.|,|:|\(|\)', temp)[0]
                temp=temp.strip()
                certificados_match_metric.append(temp)
    #print('certificados_match_metric: ', certificados_match_metric)
    certificados_match_metric = list(set(certificados_match_metric))
    
    for row in certificados_match_metric:
        row = {'id_tramite': name,
                'nombre_tramite': group['nombre_tramite'].values[0],
                'certificados': row}

        data_tramites_certificados_list.append(row)
    

In [ ]:
data_tramites_certificados_df = pd.DataFrame(data_tramites_certificados_list, columns=['id_tramite','nombre_tramite','certificados'])
data_tramites_certificados_df.head()

In [ ]:
len(data_tramites_certificados_df)

In [ ]:
output_file='results/catastro_tramites - certificados_separados.xlsx'

writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
data_tramites_certificados_df.to_excel(writer, sheet_name='Sheet1', index=False)

workbook  = writer.book
worksheet = writer.sheets['Sheet1']

format1 = workbook.add_format({'text_wrap': True})

worksheet.set_column('A:A', 10)
worksheet.set_column('B:D', 50, format1)

end_row = len(data_tramites_certificados_df.index)
end_column = len(data_tramites_certificados_df.columns)-1
header = [{'header': word} for word in data_tramites_certificados_df.columns.tolist()]

worksheet.add_table(0, 0, end_row, end_column, {'header_row': True, 'columns': header})
writer.save()